In [ ]:
import pandas as pd
import numpy as np


In [593]:
df=pd.read_csv("dirty_cafe_sales.csv")
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [594]:
new_df=df.drop_duplicates(subset=None,keep="first")

In [595]:
df.loc[df["Price Per Unit"]=="3.0","Item"].unique()

array(['Cake', 'UNKNOWN', nan, 'Juice', 'ERROR'], dtype=object)

In [596]:
df.Item.unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [597]:
df.loc[df["Item"].isin(["Coffee","Cake","Cookie",'Juice','Sandwich','Smoothie','Salad']),["Price Per Unit","Item"]]

,Price Per Unit,Item
0,2.0,Coffee
1,3.0,Cake
2,1.0,Cookie
3,5.0,Salad
4,2.0,Coffee
...,...,...
9993,4.0,Smoothie
9995,2.0,Coffee
9997,2.0,Coffee
9998,NaN,Cookie


In [598]:
df.loc[df["Item"]=="NaN","Item"]

Series([], Name: Item, dtype: object)

In [599]:
price_mapping={'2.0':"Coffee"
               , '1.0':"Cookie"
               , '5.0':"Salad"
               , '4.0':"Smoothie"
               , '1.5':"Tea"
    
}

In [600]:
df.loc[df["Item"].isin(["UNKNOWN","ERROR",np.nan]),"Item"]=df["Price Per Unit"].map(price_mapping)

In [601]:
df.loc[df["Price Per Unit"].isin(["UNKNOWN","ERROR",np.nan]),"Price Per Unit"]=df["Item"].map(price_mapping)

In [602]:
df["Item"]=df["Item"].apply(lambda x: np.random.choice(['Juice','Cake']) if x in [np.nan,"UNKNOWN","ERROR"] else x)

In [603]:
df["Price Per Unit"]=df["Price Per Unit"].apply(lambda x: "3.0" if x in [np.nan,"UNKNOWN","ERROR"] else x)

In [604]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,Juice,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,Cake,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [605]:
df["Item"]=df["Item"].astype(str)

In [606]:
df["Price Per Unit"].unique()

array(['2.0', '3.0', '1.0', '5.0', '4.0', '1.5'], dtype=object)

In [607]:
error=df.loc[df["Quantity"]=="ERROR","Quantity"].count()
un=df.loc[df["Quantity"]=="UNKNOWN","Quantity"].count()
un+error


np.int64(341)

In [608]:
df["Quantity"]=df["Quantity"].replace(["UNKNOWN","ERROR"],"0")

In [609]:
df["Quantity"]=df["Quantity"].astype(float)

In [610]:
df["Quantity"]=df["Quantity"].replace(np.nan,round(df["Quantity"].mean()))
df["Quantity"]=df["Quantity"].replace(0,round(df["Quantity"].mean()))
df["Quantity"]=df["Quantity"].astype(int)
df["Price Per Unit"]=df["Price Per Unit"].astype(float)

In [611]:
df.drop(columns="Total Spent")
df["Total Spent"]=df["Price Per Unit"]*df["Quantity"]

In [612]:
df["Transaction Date"].ffill(inplace=True)
date=df["Transaction Date"].unique()
date=list(date)
len(date)
date.remove("ERROR")
date.remove("UNKNOWN")

C:\Users\MD AZAM KHAN\AppData\Local\Temp\ipykernel_6352\2004503229.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Transaction Date"].ffill(inplace=True)


In [613]:

df["Transaction Date"].replace(["UNKNOWN","ERROR"],np.random.choice(date),inplace=True)
df["Transaction Date"]=df["Transaction Date"].astype("date32[pyarrow]")

In [614]:
df["Transaction Date"].describe

<bound method NDFrame.describe of 0       2023-09-08
1       2023-05-16
2       2023-07-19
3       2023-04-27
4       2023-06-11
           ...    
9995    2023-08-30
9996    2023-06-02
9997    2023-03-02
9998    2023-12-02
9999    2023-11-07
Name: Transaction Date, Length: 10000, dtype: date32[day][pyarrow]>

In [615]:
df["Payment Method"]=df["Payment Method"].fillna(df["Payment Method"].mode()[0])
df["Payment Method"]=df["Payment Method"].replace(["UNKNOWN","ERROR"],df["Payment Method"].mode()[0])


In [616]:
df=df.drop(columns="Location")


In [617]:
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,2023-05-16
2,TXN_4271903,Cookie,4,1.0,4.0,Credit Card,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Digital Wallet,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,2023-06-11
...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,Digital Wallet,2023-08-30
9996,TXN_9659401,Cake,3,3.0,9.0,Digital Wallet,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,2023-03-02
9998,TXN_7695629,Cookie,3,3.0,9.0,Digital Wallet,2023-12-02


In [618]:
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,2023-05-16
2,TXN_4271903,Cookie,4,1.0,4.0,Credit Card,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Digital Wallet,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,2023-06-11
...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,Digital Wallet,2023-08-30
9996,TXN_9659401,Cake,3,3.0,9.0,Digital Wallet,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,2023-03-02
9998,TXN_7695629,Cookie,3,3.0,9.0,Digital Wallet,2023-12-02


In [619]:
df.to_csv("clean_cafe_sales.csv")